# Talk to your code

#Step 01: Install All the Required Packages

In [ ]:
!pip install openai tiktoken chromadb langchain langchain_mistralai langchain-community

In [ ]:
!pip install GitPython

# Step 02: Initialize API keys

In [ ]:
import os
import sys

os.environ['MISTRAL_API_KEY'] = '<your Mistral API KEY goes here>'

# Step 03: Clone the LangChain Github Repo

In [ ]:
#!rm -rf test_repo

In [ ]:
!mkdir test_repo

In [ ]:
from git import Repo

repo_path = "/content/test_repo"

repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)

# Step 04: Load the Data from the Files

In [ ]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser


loader = GenericLoader.from_filesystem(
    repo_path+'/libs/langchain/langchain',
    glob = "**/*",
    suffixes=[".py"],
    parser = LanguageParser(
        language=Language.PYTHON, 
        parser_threshold=500
    )
)

In [ ]:
documents = loader.load()

In [ ]:
len(documents)

In [ ]:
documents[0]

# Step 05: Split the Document into Chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [ ]:
texts = documents_splitter.split_documents(documents)

In [ ]:
len(texts)

# Step 06: Download the MistralAI Embeddings

In [ ]:
from langchain_mistralai import MistralAIEmbeddings
embeddings=MistralAIEmbeddings()

# Step 07: Setting Up Chroma as out Vector Database

In [ ]:
from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')
vectordb.persist()

# Step 08: Creating an OpenAI Model Warpper

In [ ]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model_name="mistral-large-latest",
    temperature=0
)

In [ ]:
llm

In [ ]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [ ]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

# Talk to Your Code

In [ ]:
question = "How i can initialize the ReAct Agent"
result = qa(question)
print("Answer:", result['answer'])


In [ ]:
question = "What is the class hierarchy for ReActDocstoreAgent?"
result = qa(question)
print(result['answer'])